# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [22]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
# load messages dataset
messages = pd.read_csv('./raw_data/messages.csv',sep=',',quotechar='"')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [5]:
messages.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

### Messages notes
ID will be used to merge messages with categories. Original is unnecessary: we'll be doing our analysis based on the English messages. Genre can be useful as an additional feature. We'll see when we train the model.

In [3]:
# load categories dataset
categories = pd.read_csv('./raw_data/categories.csv',sep=',',quotechar='"')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### Categories notes
ID will be used to merge messages with categories. The categories field will need to be split into multiple fields based on the separator.

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

### Merge notes
Both datasets have an 'id' column. pd.merge() should work automatically.

# Checking for duplicates should come before merges and joins!

In [7]:
print('Number of rows in Messages is {}'.format(len(messages)))
print('Number of unique IDs in Messages is {}'.format(len(messages['id'].unique())))

print('Number of rows in Categories is {}'.format(len(categories)))
print('Number of unique IDs in Categories is {}'.format(len(categories['id'].unique())))

Number of rows in Messages is 26248
Number of unique IDs in Messages is 26180
Number of rows in Categories is 26248
Number of unique IDs in Categories is 26180


# We have duplicates
We don't know the nature of these duplicates. Are the rows completely the same? Maybe they have the same IDs but different data? It's unsafe nad outright bad practice to merge the datasets at this point. Let's try to clean the datasets up.

In [4]:
messages = messages.drop_duplicates()
categories = categories.drop_duplicates()

In [9]:
print('After basic duplicates drop.')
print('Number of rows in Messages is {}'.format(len(messages)))
print('Number of unique IDs in Messages is {}'.format(len(messages['id'].unique())))

print('Number of rows in Categories is {}'.format(len(categories)))
print('Number of unique IDs in Categories is {}'.format(len(categories['id'].unique())))

After basic duplicates drop.
Number of rows in Messages is 26180
Number of unique IDs in Messages is 26180
Number of rows in Categories is 26216
Number of unique IDs in Categories is 26180


## Messages are clean, Categories need more processing
There are still duplicates in categories. We should get rid of them before merging.
Let's take a look at the duplicate rows.

In [10]:
categories.groupby?

In [5]:
id_counts = categories.groupby(by=['id']).count()
id_dupl = id_counts[id_counts.values > 1]
categories[categories['id'].isin(id_dupl.index)].sort_values(by=['id']).iloc[:10,:]

,id,categories
162,202,related-1;request-1;offer-0;aid_related-1;medi...
163,202,related-1;request-1;offer-0;aid_related-1;medi...
709,862,related-0;request-0;offer-0;aid_related-0;medi...
710,862,related-1;request-0;offer-0;aid_related-0;medi...
1407,1652,related-1;request-1;offer-0;aid_related-1;medi...
1408,1652,related-1;request-1;offer-0;aid_related-1;medi...
2824,3250,related-1;request-0;offer-0;aid_related-0;medi...
2825,3250,related-1;request-1;offer-0;aid_related-0;medi...
3374,3882,related-1;request-1;offer-0;aid_related-1;medi...
3373,3882,related-1;request-1;offer-0;aid_related-1;medi...


In [25]:
categories[categories['id']==3250]

,id,categories
2824,3250,related-1;request-0;offer-0;aid_related-0;medi...
2825,3250,related-1;request-1;offer-0;aid_related-0;medi...


In [6]:
print(len(id_dupl))
print(len(id_dupl[id_dupl.values==2]))

36
36


### There are 36 different rows with the same id in categories
We have 18 IDs that have 2 variants of category labelling each. This is not perfect, but the dataset has approximately 28000 observations, so it's not a big deal. Let's just keep an arbitrary variant and move on.

In [7]:
categories = categories.drop_duplicates(subset=['id'],keep='first')

In [8]:
print('After final duplicates drop.')
print('Number of rows in Messages is {}'.format(len(messages)))
print('Number of unique IDs in Messages is {}'.format(len(messages['id'].unique())))

print('Number of rows in Categories is {}'.format(len(categories)))
print('Number of unique IDs in Categories is {}'.format(len(categories['id'].unique())))

After final duplicates drop.
Number of rows in Messages is 26180
Number of unique IDs in Messages is 26180
Number of rows in Categories is 26180
Number of unique IDs in Categories is 26180


### Now we can merge

In [31]:
# merge datasets
df = pd.merge(messages, categories)
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [32]:
print('The number of rows in the merged dataset is {}'.format(len(df)))

The number of rows in the merged dataset is 26180


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

# Notes on splitting into categories.
The str.split() approach is lazy. We have no guarantee that the categories in the first row extend across all the 26180 rows of the dataset. We need to be sure, so let's test before we split.

In [9]:
#Get column names for testing
test_cols = [x[:-2] for x in categories.iloc[0,1].split(';')]

In [10]:
#Build a testing function
def test_categories(text, test_cols):
    """
    Test if the encoded categories haven't changed
    
    Args:
    text (str): string with categories and values encoded
    test_cols (list): list of category names to check against
    
    Output:
    (bool): True if the categories and their order haven't
            changed, False otherwise
    """
    t2 = test_cols
    t1 = [x[:-2] for x in text.split(';')]
    if t1==t2:
        return True
    else:
        return False

In [11]:
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [12]:
cat_test = categories['categories'].apply(lambda x: test_categories(x, test_cols))
cat_test[~cat_test.values]

Series([], Name: categories, dtype: bool)

### We've checked that the columns and their order are the same for every row
Such tests should be done unless coding and running them is prhobitively expensive. This was simple to code and took almost no time to run.
Now we are ready to do the split. We'll do it a bit differently from what's suggested in the Notebook for readability. The result is the same though.

In [13]:
#Get column names
cols = categories.loc[0,'categories'].split(';')
cols = [x[:-2] for x in cols]
print(cols)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [14]:
#Get data
def get_cat_values(text):
    """
    This basic function takes an encoded category string
    and returns a list of 0s and 1s indicating 
    categories. It could be written as a lambda,
    but it would be less readable.
    
    Args:
    text (str): text with values encoded in them
    
    Output:
    val_list (list): a list of 0s and 1s flags
    """
    
    vals = [int(x[-1]) for x in text.split(';')]
    return vals

In [15]:
#Turn a Series of str in to a Series of lists
data = categories['categories'].apply(get_cat_values)
#Turn the Series into a list of lists
data=list(data)
#Direct cast to np.array didn't work as expected (made an array of lists)
data=np.array(data)
data.shape

(26180, 36)

In [16]:
#Make the ID column
ids = np.array(categories['id'])[np.newaxis].T

In [17]:
#Add ID to the values and to the column names list
data_full = np.hstack((data,ids))
cols_full = cols + ['id']

In [18]:
# rename the columns of `categories`

#We'll make a new categories DataFrame instead
categories_new = pd.DataFrame(data=data_full,columns=cols_full)
categories_new.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,id
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,1,0,0,1,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,7
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [ ]:
#We took care of this during step 3, because there was no need to
#make a weird DataFrame full of text, when we easily did this in one step.

#for column in categories:
#    # set each value to be the last character of the string
#    categories[column] = 
#    
#    # convert column from string to numeric
#    categories[column] = 
#categories.head()

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [19]:
#This is a weird way to do this. Let's just re-merge with the new DataFrame we made.
df = pd.merge(messages, categories_new)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

## We already did this. Duplicates should be cleaned before any joins or merges.
It's dangerous to merge or join before removing duplicates, because duplicates can and will multiply (the program infers a many-to-many operation when running into duplicates). Things can get out of hand fast.
This is why I've cleaned everything already.

In [ ]:
#Already done during step 2
# check number of duplicates


In [ ]:
#Already done during step 2
# drop duplicates


In [ ]:
#Already done during step 2
# check number of duplicates


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [24]:
engine = create_engine('sqlite:///database/InsertDatabaseName.db')
df.to_sql('categorized_comments', engine, index=False)

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.